In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
X_train = np.load('/kaggle/input/segmentation-dataset/X_train.npy')
X_val = np.load('/kaggle/input/segmentation-dataset/X_val.npy')
y_train = np.load('/kaggle/input/segmentation-dataset/Y_train.npy')
y_val = np.load('/kaggle/input/segmentation-dataset/Y_val.npy')

In [ ]:
labels = pd.read_csv('/kaggle/input/segmentation-dataset/labels_data.csv')
labels.head()

In [ ]:
def str_to_tuple(rgb_str):
    return tuple(map(int, rgb_str.strip('()').split(',')))

labels['rgb'] = labels['rgb'].apply(lambda x: str_to_tuple(x) if isinstance(x, str) else x)

id_to_rgb = dict(zip(labels['id'], labels['rgb']))

In [ ]:
def mask_to_rgb(mask, id_to_rgb):
    rgb_mask = np.zeros((mask.shape[0], mask.shape[1], 3), dtype=np.uint8)
    for class_id, color in id_to_rgb.items():
        rgb_mask[mask == class_id] = color
    
    return rgb_mask

In [ ]:
print(f'X_train shape {X_train.shape}\nY_train shape {y_train.shape}\nX_val shape {X_val.shape}\nY_val shape {y_val.shape}')

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, BatchNormalization, Dropout, Add
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
def build_segnet(input_shape=(128, 256, 3), num_classes=29):
    inputs = Input(shape=input_shape)
    
    # Encoder
    # Block 1
    x = Conv2D(64, (3, 3), padding='same', activation='relu')(inputs)
    x = BatchNormalization()(x)
    x = Conv2D(64, (3, 3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    p1 = MaxPooling2D((2, 2))(x)
    
    # Block 2 with dropout
    x = Conv2D(128, (3, 3), padding='same', activation='relu')(p1)
    x = BatchNormalization()(x)
    x = Conv2D(128, (3, 3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)  
    p2 = MaxPooling2D((2, 2))(x)

    # Block 3 with Dilated Convolutions
    x = Conv2D(256, (3, 3), padding='same', activation='relu')(p2)
    x = BatchNormalization()(x)
    x = Conv2D(256, (3, 3), dilation_rate=2, padding='same', activation='relu')(x)  # Dilated convolution
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)  
    p3 = MaxPooling2D((2, 2))(x)

    # Block 4 with Dilated Convolutions
    x = Conv2D(512, (3, 3), padding='same', activation='relu')(p3)
    x = BatchNormalization()(x)
    x = Conv2D(512, (3, 3), dilation_rate=2, padding='same', activation='relu')(x)  # Dilated convolution
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)  
    p4 = MaxPooling2D((2, 2))(x)

    # Decoder
    # Block 4
    x = UpSampling2D((2, 2))(p4)
    x = Conv2D(512, (3, 3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = Conv2D(512, (3, 3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)  

    # Block 3
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(256, (3, 3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = Conv2D(256, (3, 3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)  

    # Block 2
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(128, (3, 3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = Conv2D(128, (3, 3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)

    # Block 1
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(64, (3, 3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = Conv2D(64, (3, 3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)

    # Output layer with softmax
    outputs = Conv2D(num_classes, (1, 1), padding='same', activation='softmax')(x)

    model = Model(inputs=inputs, outputs=outputs)
    return model

In [ ]:
early_stopping = EarlyStopping(
    monitor='val_accuracy',  
    patience=5,              
    verbose=1,              
    restore_best_weights=True 
)

In [ ]:
input_shape = (128, 256, 3)  
num_classes = 29
model = build_segnet(input_shape=input_shape, num_classes=num_classes)

model.compile(optimizer=Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

In [ ]:
history = model.fit(
    X_train, y_train, 
    validation_data=(X_val, y_val),  
    epochs=200,                     
    batch_size=32,                  
    callbacks=[early_stopping]       
) 

In [ ]:
val_loss, val_acc = model.evaluate(X_val, y_val)
print(f'Validation Loss: {val_loss}')
print(f'Validation Accuracy: {val_acc}')

pred_mask = model.predict(X_val[0:1])
pred_mask = tf.argmax(pred_mask, axis=-1)  
pred_mask = pred_mask[0]  

print(pred_mask.shape)

In [ ]:
def mask_to_rgb(mask, id_to_rgb):
    height, width = mask.shape
    rgb_mask = np.zeros((height, width, 3), dtype=np.uint8)

    for class_id, color in id_to_rgb.items():
        rgb_mask[mask == class_id] = np.array(color, dtype=np.uint8) 
    
    return rgb_mask

In [ ]:
def show_image_and_colored_masks(img, true_mask_rgb, pred_mask_rgb):
    plt.figure(figsize=(15, 5))

    plt.subplot(1, 3, 1)
    plt.imshow(img)
    plt.title('Test Image')
    plt.axis('off')

    plt.subplot(1, 3, 2)
    plt.imshow(true_mask_rgb)
    plt.title('Ground Truth Mask (RGB)')
    plt.axis('off')

    plt.subplot(1, 3, 3)
    plt.imshow(pred_mask_rgb)
    plt.title('Predicted Mask (RGB)')
    plt.axis('off')

    plt.show()

In [ ]:
index = 200
test_img = X_val[index] 
true_mask = y_val[index]  


pred_mask = model.predict(test_img[np.newaxis, ...])  
pred_mask = tf.argmax(pred_mask, axis=-1)[0].numpy()  


pred_mask_rgb = mask_to_rgb(pred_mask, id_to_rgb)
true_mask_rgb = mask_to_rgb(true_mask, id_to_rgb)

show_image_and_colored_masks(test_img, true_mask_rgb, pred_mask_rgb)

In [ ]:
model.save('segnet-83,68.keras')